In [514]:

import numpy as np
import pandas as pd

In [515]:

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [516]:
df = pd.read_csv('train.csv')

In [517]:

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## **Let's Plan**

In [518]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [519]:
# Step 1 -> train/test/split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),
                                                 df['Survived'],
                                                 test_size=0.2,
                                                random_state=42)

In [520]:

X_train.sample(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
304,3,male,NaN,0,0,8.0500,S
343,2,male,25.0,0,0,13.0000,S
560,3,male,NaN,0,0,7.7500,Q
836,3,male,21.0,0,0,8.6625,S
603,3,male,44.0,0,0,8.0500,S


In [521]:

y_train.sample(5)

,Survived
381,1
413,0
102,0
587,1
675,0


In [522]:
# imputation transformer filling the missing values
# imputation transformer
a = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]), # adding col index number  instead of name
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6]) # adding col index number  instead of name
],remainder='passthrough')

In [523]:
# one hot encoding
b = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])
    # adding col index number  instead of name
],remainder='passthrough')

In [524]:
# Scaling
c = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [525]:
# Feature selection
d = SelectKBest(score_func=chi2,k=8)

In [526]:

# train the model
e = DecisionTreeClassifier()

# **Create Pipeline**

In [527]:

pipe = Pipeline([
    ('a',a),
    ('b',b),
    ('c',c),
    ('d',d),
    ('e',e)
])

In [528]:

# Alternate Syntax
pipe = make_pipeline(a,b,c,d,e)

In [529]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7e0e0367f600>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])


## **Explore the Pipeline**

In [530]:
pipe.named_steps

{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'columntransformer-2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [1, 6])]),
 'columntransformer-3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'selectkbest': SelectKBest(k=8, score_func=<function chi2 at 0x7e0e0367f600>),
 'decisiontreeclassifier': DecisionTreeClassifier()}

In [531]:
pipe.named_steps['columntransformer-1'].transformers_[0][1].statistics_

array([29.49884615])

In [532]:
pipe.named_steps['columntransformer-1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [533]:
pipe.named_steps['columntransformer-2']

ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe_sex_embarked',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 [1, 6])])

In [534]:
# Code here
pipe.named_steps['columntransformer-2'].named_transformers_['ohe_sex_embarked'].categories_

[array(['C', 'Q', 'S'], dtype=object),
 array([0.0, 4.0125, 5.0, 6.2375, 6.4375, 6.45, 6.4958, 6.75, 6.8583, 6.95,
        6.975, 7.05, 7.0542, 7.125, 7.1417, 7.225, 7.2292, 7.25, 7.3125,
        7.4958, 7.5208, 7.55, 7.6292, 7.65, 7.725, 7.7333, 7.7375, 7.7417,
        7.75, 7.775, 7.7958, 7.8, 7.8292, 7.8542, 7.8792, 7.8875, 7.8958,
        7.925, 8.0292, 8.05, 8.1125, 8.1375, 8.3, 8.3625, 8.4042, 8.5167,
        8.6542, 8.6625, 8.6833, 8.85, 9.0, 9.2167, 9.35, 9.475, 9.4833,
        9.5, 9.5875, 9.825, 9.8375, 9.8417, 10.1708, 10.4625, 10.5,
        10.5167, 11.1333, 11.2417, 11.5, 12.0, 12.275, 12.2875, 12.35,
        12.475, 12.525, 12.65, 13.0, 13.4167, 13.5, 13.7917, 13.8583,
        13.8625, 14.0, 14.1083, 14.4, 14.4542, 14.4583, 14.5, 15.0,
        15.0458, 15.05, 15.1, 15.2458, 15.5, 15.7417, 15.75, 15.85, 15.9,
        16.0, 16.1, 16.7, 17.4, 17.8, 18.0, 18.75, 18.7875, 19.2583, 19.5,
        19.9667, 20.2125, 20.25, 20.525, 20.575, 21.0, 21.075, 21.6792,
        22.025, 22.

In [535]:
pipe.named_steps['columntransformer-3']

ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))])

In [536]:
pipe.named_steps['columntransformer-3'].transformers_[0][2]

slice(0, 10, None)

In [537]:
pipe.named_steps['decisiontreeclassifier']

DecisionTreeClassifier()

In [538]:
pipe.named_steps['decisiontreeclassifier'].feature_importances_

array([0.69727746, 0.00314777, 0.1514688 , 0.0199216 , 0.01985194,
       0.04884444, 0.01999162, 0.03949637])

In [539]:
# Display Pipeline

from sklearn import set_config
set_config(display='diagram')

In [540]:
# Predict
y_pred = pipe.predict(X_test)
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0])

In [541]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464


# ***Cross Validation using Pipeline***

In [542]:
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, X_train, y_train, cv=15, scoring='accuracy').mean()*100

np.float64(63.91548463356974)

# ***GridSearch using Pipeline***

In [543]:
params = {
    'decisiontreeclassifier__max_depth': [1, 2, 3, 4, 5, None]
}

In [544]:

from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer-1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('columntransformer-2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unkno...ore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('columntransformer-3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('selectkbest',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x7e0e0367f600>)),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'decisiontreeclassifier__max_depth': [1, 2, 3, 4, 5,
                                                               None]},
             scoring='accuracy')

In [545]:
grid.best_score_

np.float64(0.6391214419383433)

In [546]:

grid.best_params_

{'decisiontreeclassifier__max_depth': 2}

# ***Exporting the Pipeline***

In [549]:

# export
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))

# **Opening the file**

In [550]:
import pickle
import numpy as np


In [551]:
p=pickle.load(open('pipe.pkl','rb'))


In [552]:
t=np.array([2,'male',31.0,0,0,10.5,'S'],dtype=object).reshape(1,7)

In [553]:
p.predict(t)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


array([0])